# Exploring the possiblities with Python package PyDy

In [1]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)
from sympy.physics.mechanics import inertia, RigidBody
from sympy import trigsimp
from sympy.physics.mechanics import KanesMethod

## Reference frames

In [2]:
inertial_frame = ReferenceFrame('I')
ship_frame = ReferenceFrame('S')

In [3]:
psi = dynamicsymbols('psi') #Ship heading
X = dynamicsymbols('X') #Ship position in inertial frame
Y = dynamicsymbols('Y') #Ship position in inertial frame

In [4]:
ship_frame.orient(inertial_frame, 'Axis', (psi, inertial_frame.z))

In [5]:
#phi = dynamicsymbols('phi') #Ship roll
#theta = dynamicsymbols('theta') #Ship pitch

In [6]:
ship_frame.dcm(inertial_frame)

Matrix([
[ cos(psi), sin(psi), 0],
[-sin(psi), cos(psi), 0],
[        0,        0, 1]])

## Define some points

In [7]:
starting_point = Point('Start')
ship_origo = Point('O')
ship_mass_center = Point('c_G')
x_cg = symbols('x_cg')
y_cg = symbols('y_cg')
z_cg = symbols('z_cg')

ship_mass_center.set_pos(ship_origo, x_cg*ship_frame.x + y_cg*ship_frame.y + z_cg*ship_frame.z)

## Kinematical Differential Equations

In [8]:
r = dynamicsymbols('r') # Yaw rate
U = dynamicsymbols('U') # Ship speed in inertial frame
V = dynamicsymbols('V') # Ship speed in inertial frame


In [9]:
kinematical_differential_equations = [r - psi.diff(),
                                      U - X.diff(),
                                      V - Y.diff()]
kinematical_differential_equations

[r - psi', U - X', V - Y']

## Angular Velocities

In [10]:
ship_frame.set_ang_vel(inertial_frame,r*inertial_frame.z)

In [11]:
ship_frame.ang_vel_in(inertial_frame)

r*I.z

## Linear Velocities

In [12]:


starting_point.set_vel(inertial_frame, 0)

ship_origo.set_vel(inertial_frame,U*inertial_frame.x + V*inertial_frame.y)

ship_origo.vel(inertial_frame)

U*I.x + V*I.y

In [13]:
ship_mass_center.v2pt_theory(ship_origo, inertial_frame, ship_frame)

U*I.x + V*I.y - y_cg*r*S.x + x_cg*r*S.y

## Mass

In [14]:
ship_mass = symbols('m_S')

## Inertia

In [15]:
Izz = symbols('I_zz')

In [16]:
ship_dyadic = inertia(ship_frame, 0, 0, Izz)

In [17]:
ship_dyadic

I_zz*(S.z|S.z)

In [18]:
ship_dyadic.to_matrix(ship_frame)

Matrix([
[0, 0,    0],
[0, 0,    0],
[0, 0, I_zz]])

In [19]:
ship_central_inertia = (ship_dyadic, ship_mass_center)

In [20]:
ship = RigidBody('Ship', ship_mass_center, ship_frame,
                      ship_mass, ship_central_inertia)

## Equations of Motion

In [33]:
coordinates = [psi,X, Y]
coordinates

[psi, X, Y]

In [34]:
speeds = [r,U, V]
speeds

[r, U, V]

In [35]:
kane = KanesMethod(inertial_frame, coordinates, speeds, kinematical_differential_equations)

In [36]:
dummy_force = symbols('force')
ship_force_vector = dummy_force * ship_frame.y

In [37]:
loads = [ship_force_vector]

In [38]:
bodies = [ship]

In [39]:
fr, frstar = kane.kanes_equations(loads, bodies)

TypeError: 'RigidBody' object is not iterable